<a href="https://colab.research.google.com/github/krizely/mcba_usb/blob/master/pois.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import POIs from OSM

Run the full code (Execution/Execute All). Look at the notepad footer for user interface.

For long GPX files, like French Divide, the process can take a long time. Try to split the GPX. The result will be a KML file to import into Google Map. The script is working fine with [727 routes](https://tcrouzet.com/i727/).

The Google Places API is paid and relatively limited. Therefore, I only use the POIs from the OSM map. It's not exhaustive, but it's a very good starting point.

By Thierry Crouzet : [tcrouzet.com](https://tcrouzet.com/) (contact-me if you find bugs or have improuvement suggestions).

## Init Code

In [ ]:
from typing_extensions import TypeVarTuple
import geopandas as gpd #?
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript
from google.colab import files
import requests
import xml.etree.ElementTree as ET
import math
import requests
import time
import urllib.parse

POI_TAGS = {
    'amenity': ['drinking_water', 'restaurant', 'fountain', 'toilets', 'cafe', 'bar', 'shelter'],
    'shop': ['bakery', 'grocery', 'supermarket', 'pastry'],
    'tourism': ['camp_site', 'guest_house', 'hotel']
}

FORM_TAGS = {
    'Water': ['drinking_water', 'fountain', 'toilets'],
    'Shelter': ['shelter'],
    'Food': ['bakery', 'grocery', 'supermarket', 'pastry', 'restaurant', 'cafe', 'bar'],
    'Accommodation': ['camp_site', 'guest_house', 'hotel']
}

DISTANCE_LIMIT = 250

# Define global variables
gpx_tree = []
selected_pois = [description for description in FORM_TAGS]
export_name = ''
start_time = None

## KMK Style

You can edit the file hearder. Not easy.

In [ ]:
KML_STYLE = """<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2">
  <Document>
    <name>POIs by Crouzet</name>
    <Style id="icon-1500-000000-nodesc-normal">
      <IconStyle>
        <color>ff000000</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>0</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <Style id="icon-1500-000000-nodesc-highlight">
      <IconStyle>
        <color>ff000000</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>1</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <StyleMap id="icon-1500-000000-nodesc">
      <Pair>
        <key>normal</key>
        <styleUrl>#icon-1500-000000-nodesc-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#icon-1500-000000-nodesc-highlight</styleUrl>
      </Pair>
    </StyleMap>
    <Style id="icon-1508-9C27B0-nodesc-normal">
      <IconStyle>
        <color>ffb0279c</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>0</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <Style id="icon-1508-9C27B0-nodesc-highlight">
      <IconStyle>
        <color>ffb0279c</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>1</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <StyleMap id="icon-1508-9C27B0-nodesc">
      <Pair>
        <key>normal</key>
        <styleUrl>#icon-1508-9C27B0-nodesc-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#icon-1508-9C27B0-nodesc-highlight</styleUrl>
      </Pair>
    </StyleMap>
    <Style id="icon-1517-F9A825-nodesc-normal">
      <IconStyle>
        <color>ff25a8f9</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>0</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <Style id="icon-1517-F9A825-nodesc-highlight">
      <IconStyle>
        <color>ff25a8f9</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>1</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <StyleMap id="icon-1517-F9A825-nodesc">
      <Pair>
        <key>normal</key>
        <styleUrl>#icon-1517-F9A825-nodesc-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#icon-1517-F9A825-nodesc-highlight</styleUrl>
      </Pair>
    </StyleMap>
    <Style id="icon-1522-0F9D58-nodesc-normal">
      <IconStyle>
        <color>ff589d0f</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>0</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <Style id="icon-1522-0F9D58-nodesc-highlight">
      <IconStyle>
        <color>ff589d0f</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>1</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <StyleMap id="icon-1522-0F9D58-nodesc">
      <Pair>
        <key>normal</key>
        <styleUrl>#icon-1522-0F9D58-nodesc-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#icon-1522-0F9D58-nodesc-highlight</styleUrl>
      </Pair>
    </StyleMap>
    <Style id="icon-1535-FFD600-nodesc-normal">
      <IconStyle>
        <color>ff00d6ff</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>0</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <Style id="icon-1535-FFD600-nodesc-highlight">
      <IconStyle>
        <color>ff00d6ff</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>1</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <StyleMap id="icon-1535-FFD600-nodesc">
      <Pair>
        <key>normal</key>
        <styleUrl>#icon-1535-FFD600-nodesc-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#icon-1535-FFD600-nodesc-highlight</styleUrl>
      </Pair>
    </StyleMap>
    <Style id="icon-1535-FFEA00-nodesc-normal">
      <IconStyle>
        <color>ff00eaff</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>0</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <Style id="icon-1535-FFEA00-nodesc-highlight">
      <IconStyle>
        <color>ff00eaff</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>1</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <StyleMap id="icon-1535-FFEA00-nodesc">
      <Pair>
        <key>normal</key>
        <styleUrl>#icon-1535-FFEA00-nodesc-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#icon-1535-FFEA00-nodesc-highlight</styleUrl>
      </Pair>
    </StyleMap>
    <Style id="icon-1577-F9A825-nodesc-normal">
      <IconStyle>
        <color>ff25a8f9</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>0</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <Style id="icon-1577-F9A825-nodesc-highlight">
      <IconStyle>
        <color>ff25a8f9</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>1</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <StyleMap id="icon-1577-F9A825-nodesc">
      <Pair>
        <key>normal</key>
        <styleUrl>#icon-1577-F9A825-nodesc-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#icon-1577-F9A825-nodesc-highlight</styleUrl>
      </Pair>
    </StyleMap>
    <Style id="icon-1578-F9A825-nodesc-normal">
      <IconStyle>
        <color>ff25a8f9</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>0</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <Style id="icon-1578-F9A825-nodesc-highlight">
      <IconStyle>
        <color>ff25a8f9</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>1</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <StyleMap id="icon-1578-F9A825-nodesc">
      <Pair>
        <key>normal</key>
        <styleUrl>#icon-1578-F9A825-nodesc-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#icon-1578-F9A825-nodesc-highlight</styleUrl>
      </Pair>
    </StyleMap>
    <Style id="icon-1602-9C27B0-normal">
      <IconStyle>
        <color>ffb0279c</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>0</scale>
      </LabelStyle>
    </Style>
    <Style id="icon-1602-9C27B0-highlight">
      <IconStyle>
        <color>ffb0279c</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>1</scale>
      </LabelStyle>
    </Style>
    <StyleMap id="icon-1602-9C27B0">
      <Pair>
        <key>normal</key>
        <styleUrl>#icon-1602-9C27B0-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#icon-1602-9C27B0-highlight</styleUrl>
      </Pair>
    </StyleMap>
    <Style id="icon-1602-9C27B0-nodesc-normal">
      <IconStyle>
        <color>ffb0279c</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>0</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <Style id="icon-1602-9C27B0-nodesc-highlight">
      <IconStyle>
        <color>ffb0279c</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>1</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <StyleMap id="icon-1602-9C27B0-nodesc">
      <Pair>
        <key>normal</key>
        <styleUrl>#icon-1602-9C27B0-nodesc-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#icon-1602-9C27B0-nodesc-highlight</styleUrl>
      </Pair>
    </StyleMap>
    <Style id="icon-1634-FFEA00-nodesc-normal">
      <IconStyle>
        <color>ff00eaff</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>0</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <Style id="icon-1634-FFEA00-nodesc-highlight">
      <IconStyle>
        <color>ff00eaff</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>1</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <StyleMap id="icon-1634-FFEA00-nodesc">
      <Pair>
        <key>normal</key>
        <styleUrl>#icon-1634-FFEA00-nodesc-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#icon-1634-FFEA00-nodesc-highlight</styleUrl>
      </Pair>
    </StyleMap>
    <Style id="icon-1651-F9A825-nodesc-normal">
      <IconStyle>
        <color>ff25a8f9</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>0</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <Style id="icon-1651-F9A825-nodesc-highlight">
      <IconStyle>
        <color>ff25a8f9</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>1</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <StyleMap id="icon-1651-F9A825-nodesc">
      <Pair>
        <key>normal</key>
        <styleUrl>#icon-1651-F9A825-nodesc-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#icon-1651-F9A825-nodesc-highlight</styleUrl>
      </Pair>
    </StyleMap>
    <Style id="icon-1660-FF5252-normal">
      <IconStyle>
        <color>ff5252ff</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>0</scale>
      </LabelStyle>
    </Style>
    <Style id="icon-1660-FF5252-highlight">
      <IconStyle>
        <color>ff5252ff</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>1</scale>
      </LabelStyle>
    </Style>
    <StyleMap id="icon-1660-FF5252">
      <Pair>
        <key>normal</key>
        <styleUrl>#icon-1660-FF5252-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#icon-1660-FF5252-highlight</styleUrl>
      </Pair>
    </StyleMap>
    <Style id="icon-1660-FF5252-nodesc-normal">
      <IconStyle>
        <color>ff5252ff</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>0</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <Style id="icon-1660-FF5252-nodesc-highlight">
      <IconStyle>
        <color>ff5252ff</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>1</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <StyleMap id="icon-1660-FF5252-nodesc">
      <Pair>
        <key>normal</key>
        <styleUrl>#icon-1660-FF5252-nodesc-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#icon-1660-FF5252-nodesc-highlight</styleUrl>
      </Pair>
    </StyleMap>
    <Style id="icon-1703-0288D1-nodesc-normal">
      <IconStyle>
        <color>ffd18802</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>0</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <Style id="icon-1703-0288D1-nodesc-highlight">
      <IconStyle>
        <color>ffd18802</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>1</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <StyleMap id="icon-1703-0288D1-nodesc">
      <Pair>
        <key>normal</key>
        <styleUrl>#icon-1703-0288D1-nodesc-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#icon-1703-0288D1-nodesc-highlight</styleUrl>
      </Pair>
    </StyleMap>
    <Style id="icon-1703-4186F0-nodesc-normal">
      <IconStyle>
        <color>fff08641</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>0</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <Style id="icon-1703-4186F0-nodesc-highlight">
      <IconStyle>
        <color>fff08641</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>1</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <StyleMap id="icon-1703-4186F0-nodesc">
      <Pair>
        <key>normal</key>
        <styleUrl>#icon-1703-4186F0-nodesc-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#icon-1703-4186F0-nodesc-highlight</styleUrl>
      </Pair>
    </StyleMap>
    <Style id="icon-1765-9C27B0-nodesc-normal">
      <IconStyle>
        <color>ffb0279c</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>0</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <Style id="icon-1765-9C27B0-nodesc-highlight">
      <IconStyle>
        <color>ffb0279c</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>1</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <StyleMap id="icon-1765-9C27B0-nodesc">
      <Pair>
        <key>normal</key>
        <styleUrl>#icon-1765-9C27B0-nodesc-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#icon-1765-9C27B0-nodesc-highlight</styleUrl>
      </Pair>
    </StyleMap>
    <Style id="icon-1841-0F9D58-nodesc-normal">
      <IconStyle>
        <color>ff589d0f</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>0</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <Style id="icon-1841-0F9D58-nodesc-highlight">
      <IconStyle>
        <color>ff589d0f</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>1</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <StyleMap id="icon-1841-0F9D58-nodesc">
      <Pair>
        <key>normal</key>
        <styleUrl>#icon-1841-0F9D58-nodesc-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#icon-1841-0F9D58-nodesc-highlight</styleUrl>
      </Pair>
    </StyleMap>
    <Style id="icon-1841-7CB342-nodesc-normal">
      <IconStyle>
        <color>ff42b37c</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>0</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <Style id="icon-1841-7CB342-nodesc-highlight">
      <IconStyle>
        <color>ff42b37c</color>
        <scale>1</scale>
        <Icon>
          <href>https://www.gstatic.com/mapspro/images/stock/503-wht-blank_maps.png</href>
        </Icon>
      </IconStyle>
      <LabelStyle>
        <scale>1</scale>
      </LabelStyle>
      <BalloonStyle>
        <text><![CDATA[<h3>$[name]</h3>]]></text>
      </BalloonStyle>
    </Style>
    <StyleMap id="icon-1841-7CB342-nodesc">
      <Pair>
        <key>normal</key>
        <styleUrl>#icon-1841-7CB342-nodesc-normal</styleUrl>
      </Pair>
      <Pair>
        <key>highlight</key>
        <styleUrl>#icon-1841-7CB342-nodesc-highlight</styleUrl>
      </Pair>
    </StyleMap>
  </Document>
</kml>
"""

## Progress functions

In [ ]:
def create_progress_bar():
    display(HTML("""
        <div style="width: 100%; background-color: #f0f0f0">
            <div id="progress-bar" style="width: 0%; background-color: #b0e57c; text-align: center">
                0%
            </div>
        </div>
        <script>
        function update_progress(progress) {
            var bar = document.getElementById('progress-bar');
            bar.style.width = progress + '%';
            bar.innerText = progress + '%';
        }
        </script>
    """))

#def update_progress(progress):
#    progress=round(progress)
#    display(Javascript(f"update_progress({progress})"))

def update_progress(progress,progress_max,progress_fact,progress_min=0):
    progress=round(progress_min+(progress/progress_max)*progress_fact)
    display(Javascript(f"update_progress({progress})"))


def print_duration(message):
    global start_time
    duration = time.time() - start_time
    print(message, f" {duration:.2f} seconds")

## Functions

In [ ]:
def handle_upload(change):
    global gpx_tree,export_name,start_time

    try:

      start_time = time.time()
      print_duration("Uploading starded…")
      uploaded_file = change['owner']
      gpx_file_content = uploaded_file.value
      gpx_file_name = uploaded_file.metadata[0]['name']
      print(f"GPX file {gpx_file_name} loaded")
      export_name = gpx_file_name.replace(".gpx", "_pois.kml")

      gpx_file_data = next(iter(gpx_file_content.values()))
      gpx_file_bytes = gpx_file_data['content']
      gpx_file_bytes_decoded = gpx_file_bytes.decode('utf-8')
      gpx_tree = ET.ElementTree(ET.fromstring(gpx_file_bytes_decoded))
      print_duration("Uploading ended… You can export!")

      return True

    except Exception as e:
        print(f"Error: {e}")
        return False

def on_checkbox_change(change):
    return True
    if change['type'] == 'change' and change['name'] == 'value':
        print(f"Checkbox {change['owner'].description} changed to {change['new']}")


# Function to handle button clicks
def on_button_click(button):
    global gpx_tree,selected_pois,export_name,start_time,overpass_result

    if not gpx_tree or not list(gpx_tree.getroot()):
        print("You need first to upload a gpx file")
        return False

    start_time = time.time()

    selected_pois = []
    # Collecter les catégories de POI sélectionnées à partir des cases à cocher
    selected_categories = [checkbox.description for checkbox in checkboxes if checkbox.value]
    # Ajouter les valeurs correspondantes de POI_TAGS à selected_pois
    for category in selected_categories:
        selected_pois.extend(FORM_TAGS[category])

    print_duration("Analyse started…")
    print(f'Selected POIs: {selected_pois}')
    pois = overpass()

    #print(pois)
    create_points(pois,export_name)


# Look for POIs near de route
def overpass():
  global gpx_tree,selected_pois

  min_lat = 90
  max_lat = -90
  min_lon = 180
  max_lon = -180

  print("Find bounding box…")
  namespace = "{http://www.topografix.com/GPX/1/1}"
  trkpt_count = len(list(gpx_tree.iter(f'{namespace}trkpt')))

  i=0
  for trkpt in gpx_tree.iter(f'{namespace}trkpt'):
      lat = float(trkpt.get('lat'))
      lon = float(trkpt.get('lon'))

      min_lat = min(min_lat, lat)
      max_lat = max(max_lat, lat)
      min_lon = min(min_lon, lon)
      max_lon = max(max_lon, lon)

      i+=1
      update_progress(i, trkpt_count, 50)

      #print(f'Point: lat={lat}, lon={lon}, min_lat={min_lat}, max_lat={max_lat}, min_lon={min_lon}, max_lon={max_lon}')

  #print(f'Bounding box: ({min_lat},{min_lon},{max_lat},{max_lon})')

  print_duration("Overpass query…")
  overpass_query = '[out:json];('
  #for value in selected_pois:
  #    for key, values in POI_TAGS.items():
  #        if value in values:
  #            overpass_query += f'node["{key}"="{value}"]({min_lat},{min_lon},{max_lat},{max_lon});'

  for key, values in POI_TAGS.items():
      inside = ""
      for value in values:
          if value in selected_pois:
              if len(inside)>0:
                  inside+='|'
              inside+=value
      overpass_query += f'node["{key}"~"{inside}"]({min_lat},{min_lon},{max_lat},{max_lon});'

  overpass_query += ');out;'
  #print(f'Overpass: {overpass_query}')
  overt_url='https://overpass-turbo.eu/?Q='+urllib.parse.quote_plus(overpass_query)+";"
  print(overt_url)

  overpass_url = 'https://overpass-api.de/api/interpreter'
  response = requests.post(overpass_url, data={'data': overpass_query})
  if response.status_code != 200:
    raise Exception(f'Error fetching data from Overpass API: {response.status_code}')
  result = response.json()
  #print(result)

  print_duration("Checking if each point is within the bounding box…")
  pois = []
  for element in result['elements']:
      lat = element['lat']
      lon = element['lon']

      if min_lat <= lat <= max_lat and min_lon <= lon <= max_lon:
          pois.append(element)

  #print(pois)
  return pois

def haversine_distance(lat1, lon1, lat2, lon2):
    earth_radius = 6371000

    delta_lat = math.radians(lat2 - lat1)
    delta_lon = math.radians(lon2 - lon1)

    a = (math.sin(delta_lat / 2) ** 2 +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(delta_lon / 2) ** 2)

    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = earth_radius * c

    return distance

def calculate_distance(gpx_tree, latitude, longitude):
    distance = float('inf')

    for trkpt in gpx_tree.iter(tag='{http://www.topografix.com/GPX/1/1}trkpt'):
        point_latitude = float(trkpt.get('lat'))
        point_longitude = float(trkpt.get('lon'))

        # print(f"{point_latitude} {point_longitude} {latitude} {longitude}")

        current_distance = haversine_distance(point_latitude, point_longitude, latitude, longitude)

        if current_distance < distance:
            distance = current_distance

    return round(distance)

def create_points(pois,file_name):

  print(f'Building {file_name}')

  root = ET.fromstring(KML_STYLE)
  namespace = {'ns': root.tag.split('}')[0][1:]}
  document = root.find('./ns:Document', namespaces=namespace)
  #print(document)

  count_faraway = 0
  i = 0
  pois_count = len(pois)

  for poi in pois:

      i+=1
      update_progress(i,pois_count, 50, 50)

      lat = poi['lat']
      lon = poi['lon']
      tags = poi['tags']

      style = ""
      name = ""
      point_type = ""
      if "shop" in tags:
          if tags["shop"] == "bakery":
              name = "Boulangerie"
              style = '#icon-1578-F9A825-nodesc'
          elif tags["shop"] == "pastry":
              name = "Pâtisserie"
              style = '#icon-1578-F9A825-nodesc'
          elif tags["shop"] == "grocery":
              name = "Épicerie"
              style = '#icon-1578-F9A825-nodesc'
          elif tags["shop"] == "supermarket":
              name = "Supermarcher"
              style = '#icon-1578-F9A825-nodesc'
      elif "amenity" in tags:
          if tags["amenity"] in ["drinking_water", "fountain", "toilets"]:
              name = "Eau"
              style = '#icon-1703-0288D1-nodesc'
          elif tags["amenity"] == "restaurant":
              name = "Restaurant"
              style = '#icon-1577-F9A825-nodesc'
          elif tags["amenity"] in ["cafe", "bar"]:
              name = "Bar"
              style = '#icon-1577-F9A825-nodesc'
          elif tags["amenity"] == "shelter":
              name = "Shelter"
              style = '#icon-1508-9C27B0-nodesc'
      elif "tourism" in tags:
          if tags["tourism"] in ["guest_house", "hotel"]:
              name = "Hebergement"
              style = '#icon-1602-9C27B0-nodesc'
          elif tags["tourism"] == "camp_site":
              name = "Camping"
              style = '#icon-1765-9C27B0-nodesc'

      if not name:
          continue

      if "name" in tags and tags["name"]:
          name = f'{name}-{tags["name"]}'
      name = name.replace("&", "")

      distance = calculate_distance(gpx_tree, lat, lon)
      if distance > DISTANCE_LIMIT:
          count_faraway += 1
          continue

      # Add Placemark element to Document
      placemark = ET.SubElement(document, 'Placemark')

      # Add Point element to Placemark
      point = ET.SubElement(placemark, 'Point')

      # Add coordinates element to Point
      coordinates = ET.SubElement(point, 'coordinates')
      coordinates.text = f'{lon},{lat}'

      # Add name element to Placemark
      name_elem = ET.SubElement(placemark, 'name')
      name_elem.text = name

      # Add styleUrl element to Placemark if style is not empty
      if style:
          style_url = ET.SubElement(placemark, 'styleUrl')
          style_url.text = style

  xml_string = ET.tostring(root, encoding='utf-8', method='xml').decode('utf-8')
  xml_string = xml_string.replace('ns0:', '').replace(':ns0', '')

  print_duration("Downloading…")
  with open(file_name, 'w') as file:
    file.write(xml_string)
  files.download(file_name)
  return True


## Main Code

In [ ]:
# Create widgets
gpx_upload = widgets.FileUpload(
    accept='.gpx',
    multiple=False
)

# Set up event handlers
gpx_upload.observe(handle_upload, names='value')

# Display widgets

# Create and display checkboxes
checkboxes = []
for key in FORM_TAGS:
    checkbox = widgets.Checkbox(value=True, description=key)
    checkbox.observe(on_checkbox_change)
    checkboxes.append(checkbox)

# Group checkboxes in a VBox and display
checkbox_group = widgets.VBox(children=checkboxes)

# Create and display button
process_button = widgets.Button(description='Export POIs')
process_button.on_click(on_button_click)

## Display

In [ ]:
#Display
display(HTML("<h3>Upload a GPX file:</h3>"))
display(gpx_upload)
display(HTML("<h3>POIs to import:</h3>"))
display(checkbox_group)
display(process_button)
create_progress_bar()

FileUpload(value={}, accept='.gpx', description='Upload')

Button(description='Export POIs', style=ButtonStyle())

You need first to upload a gpx file
Uploading starded…  0.00 seconds
GPX file i727_start.gpx loaded
Uploading ended… You can export!  0.01 seconds
Analyse started…  0.00 seconds
Selected POIs: ['drinking_water', 'fountain', 'toilets', 'shelter', 'bakery', 'grocery', 'supermarket', 'pastry', 'restaurant', 'cafe', 'bar', 'camp_site', 'guest_house', 'hotel']
Find bounding box…


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloading…  4.64 seconds


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>